<a href="https://colab.research.google.com/github/alhanoufalh/Hanouf/blob/main/Copy_of_Task2_Text_Classification_ar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Classification Exam

Welcome to the Text Classification Practical Exam. In this exam, you will be tasked with building, training, and evaluating an NLP model to classify text data. You are provided with a labeled dataset containing both the text and its corresponding class labels.

Your objective is to develop a model that accurately predicts the class of the given text. Make sure to follow best practices in data preprocessing, model selection, and evaluation to achieve optimal results.

Good luck!
___

# Install and Import Needed Libraries

You can use `pyarabic` or any other library to pre-process and clean the Arabic text.

In [50]:
 pip install pyarabic

# Download the Dataset

Please note that you are allowed to take a subset of this dataset, the reason for that is it might take a long time to train the model on.

In [46]:
!kaggle datasets download -d khaledzsa/sanad
!unzip sanad.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/sanad
License(s): unknown
sanad.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  sanad.zip
replace sanad.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [47]:
import pandas as pd

In [48]:
df = pd.read_csv('sanad.csv')

# Data Exploration

Before diving into preprocessing and model building, it’s important to first explore the dataset to understand its structure, distribution, and key characteristics. This step will help you gain insights into the data and guide your decisions in subsequent steps. Here’s what to consider:

1. **Inspect the Data**:
   Start by looking at the first few rows of the dataset to get a sense of its structure. Check the columns, data types, and a few sample entries. This helps to ensure that the data is loaded correctly and gives you an initial overview of the content.

2. **Check for Missing Values**:
   Identify if there are any missing values in the dataset.

3. **Distribution of Labels**:
   Examine the distribution of the target labels (classes).

4. **Text Data Characteristics (Bonus)**:
   Analyze the length of the text data. It is useful to calculate the number of words or characters in each text sample to understand how long the texts are. This will help you set a suitable `max_length` for tokenization and padding later. You can plot a histogram of text lengths to visualize the distribution.

5. **Common Words and Vocabulary (Bonus)**:
   Explore the most frequent words in the text data.

In [51]:
df.head()

,text,label
0,https://example.com/resource/الشاٌرقة -ْ محمِد...,Culture
1,https://example.com/resource/اَنِطٌلقّتَ ٍفٍيّ...,Culture
2,https://example.com/resource/أُقيًمٌتِ مِساءُ ...,Culture
3,https://example.com/resource/بٍاسُمةَ يًوٌنٍس ...,Culture
4,https://example.com/resource/قُرر اَتحِاد اًلْ...,Culture


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39885 entries, 0 to 39884
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    39885 non-null  object
 1   label   39885 non-null  object
dtypes: object(2)
memory usage: 623.3+ KB


In [8]:
df.sample(10)

,text,label
19689,https://example.com/resource/بُلغ عِدٌد الٍحًج...,Politics
17290,https://example.com/resource/وٍجُهٍ ٍصًاحبّ ُا...,Medical
3834,https://example.com/resource/اًلِشارقُةْ: «ّاُ...,Culture
5076,https://example.com/resource/يِشًاُركِ بًيت ٌا...,Culture
19935,https://example.com/resource/أكٍدتَ كتاُئِب عُ...,Politics
32217,https://example.com/resource/اتهِمْ بيَرُنيً إ...,Sports
35752,https://example.com/resource/أُكٌدِ نٌاصر ِبنْ...,Tech
9666,https://example.com/resource/أَعُلنْتَ شّركِة ...,Finance
13047,https://example.com/resource/الُشاْرٍقّة:ُ «َا...,Medical
21680,https://example.com/resource/أوٍقفْتَ الْسَلطّ...,Politics


In [9]:
df.sample(10)

,text,label
12834,https://example.com/resource/بُدأِتٌ هًيُئْةّ ...,Medical
36874,https://example.com/resource/قّررت شرْكْةْ اَت...,Tech
14951,https://example.com/resource/دْبّيَ: ْإيٌمان ع...,Medical
31836,https://example.com/resource/اُزّداد ٍوّضّع ٍا...,Sports
6429,https://example.com/resource/دبيٌ «َالْخليّج»ُ...,Finance
29378,https://example.com/resource/أعٍلن الَاتٌحَاد ...,Sports
18702,https://example.com/resource/قضى اِلجيشً الجزِ...,Politics
13912,https://example.com/resource/رًأُس ِالخيٍمةّ -...,Medical
4176,https://example.com/resource/صدِر الّعٍددً اَل...,Culture
31849,https://example.com/resource/انِضُمً ُاٍلسبَاٍ...,Sports


# Data Cleaning and Preprocessing

In this section, we will focus on cleaning and filtering the dataset, preparing it for the text classification task. We will implement the following steps:

1. **Remove missing values**:
   First, we eliminate any rows with missing values to ensure the dataset is complete and consistent.

2. **Filter by text length (Bonus)**:
   To maintain a uniform dataset, we will filter the text samples by a specified word count range. This ensures that the texts are neither too short to lack context nor too long to introduce unnecessary complexity.

3. **Arabic stopwords loading**:
   We load a list of Arabic stopwords to filter out commonly used but contextually insignificant words. This is an important step for improving the performance of the model, as stopwords do not contribute valuable information.

4. **Text cleaning**:
   We apply a series of text cleaning steps to standardize and simplify the text data. This involves:
   - **Removing links (URLs)**: Any URLs present in the text are removed as they are not meaningful for classification purposes.
   - **Removing special characters and punctuation**: This step removes any non-alphabetical characters, ensuring the text only contains meaningful words.
   - **Removing Arabic diacritics (Tashkeel) and elongated letters (Tatweel)**: Diacritical marks and elongated letters are stripped out to standardize the text.
   - **Removing Arabic stopwords**: Words that are part of the stopwords list are removed, as they do not add value to the classification task.
   - **Stemming or Lemmmatization**: Either stemming or lemmatization is applied to reduce words to their root or base form.
   - **Normalizing Hamza**: Any variation of the Hamza character is normalized for consistency.

   **Note:** Most of these you can do using the library [PyArabic](https://pyarabic.readthedocs.io/ar/latest/README.html#features)

5. **Final cleanup**:
   Apply the cleanup function to the feature column.

By following these steps, the text will be cleaned, filtered, and ready for tokenization!

In [52]:
df.isna().sum()

,0
text,0
label,0


In [53]:
df.duplicated().sum()

0

In [54]:
df['length'] = df['text'].str.len()
df = df[df['label'] != 'Mixed']

In [55]:
df['label'].value_counts()

,count
label,
Politics,6334
Medical,6292
Culture,6098
Finance,6095
Tech,5979
Sports,5859
Religion,3228


In [56]:
from sklearn.model_selection import train_test_split

train_val, test = train_test_split(df, test_size=0.2, random_state=42)

train, val = train_test_split(train_val, test_size=0.2, random_state=42)

In [57]:
df.shape

(39885, 3)

# Tokenization, Padding, and Data Splitting

In this step, we will prepare the text data for input into a model by converting the text into numerical sequences, padding them to a uniform length, and splitting the dataset into training and testing sets. Here's an overview of the steps involved:

1. **Tokenization**:
   We use a tokenizer to convert the cleaned text into numerical sequences. You can use `Tokenizer` tokenizer from `tensorflow.keras.preprocessing.text` package or any other tokenizer you like.

2. **Text to sequences**:
   After fitting the tokenizer on the cleaned text, we transform each text into a sequence of numbers, where each number corresponds to a token (word) in the text.

3. **Padding the sequences**:
   Since different texts may vary in length, we pad the sequences to ensure they all have the same length.

4. **Label encoding**:
   The labels (target values) also need to be converted into numerical form if they are not encoded.

5. **Train-test split**:
   The dataset is divided into training and testing sets. We allocate 80% of the data for training the model and reserve 20% for testing its performance.
   
   - The **training data** consists of the padded sequences used to train the model.
   - The **training labels** are the encoded labels corresponding to the training data.
   - The **testing data** is used to assess the model’s performance after training.
   - The **testing labels** are the encoded labels corresponding to the testing data.

6. **Data shape confirmation**:
   After splitting the data, we print the shape (dimensions) of both the training and testing sets to confirm that the data is properly divided and formatted.

By the end of this step, the text data will be transformed into padded numerical sequences, the labels will be encoded, and the data will be split into training and testing sets for model development and evaluation.

In [58]:
train

,text,label,length
38617,"https://example.com/resource/دِبُي ٍ-ً ""ًاَلخٍ...",Tech,2037
30087,https://example.com/resource/فازٌ ْمنُتّخب سلَ...,Sports,644
11982,https://example.com/resource/أبٍوَظبيَ «ْاِلخل...,Finance,3280
17754,"https://example.com/resource/دٍبْي ُ- ""الخَليج...",Medical,2528
2420,https://example.com/resource/دبّيٌ ْ-ّ ّمحمدٍ ...,Culture,4979
...,...,...,...
20281,"https://example.com/resource/اٍلُجزائَرّ -ُ ""ٌ...",Politics,1370
37067,https://example.com/resource/كشًفّ ً«ّمُعهٌد ْ...,Tech,3915
15503,https://example.com/resource/نًظمٍت َطَالْباْت...,Medical,3212
35326,https://example.com/resource/أٍعُلٍنّت ّسُوني ...,Tech,1502


In [59]:
from pyarabic.araby import strip_tashkeel, strip_tatweel, normalize_hamza
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [60]:
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
import re

st = ISRIStemmer()

arabic_stopwords = set(stopwords.words('arabic'))

In [61]:
def unique_words(data):
    text = data['text'].str.cat(sep=' ')
    return set(text.split())

In [62]:
def clean_text(text):
    # text = text.strip()
    text = re.sub(r'https?\S+|www\S+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = strip_tashkeel(text)
    text = strip_tatweel(text)

    text = text.split()
    text = [word for word in text if word not in arabic_stopwords]

In [63]:
from nltk.stem import ISRIStemmer

stemmer = ISRIStemmer()

In [64]:


train['text'] = train['text'].apply(lambda text: " ".join([stemmer.stem(word) for word in text.split()]))

In [65]:
def clean_dataframe(data):
    data = data[(data['length'] >= 10) & (data['length'] <= 500)]
    return data


train = clean_dataframe(train)
train['text'] = train['text'].apply(clean_text)
val = clean_dataframe(val)
val['text'] = val['text'].apply(clean_text)
test = clean_dataframe(test)
test['text'] = test['text'].apply(clean_text)

<ipython-input-65-bb8034248a7e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['text'] = train['text'].apply(clean_text)
<ipython-input-65-bb8034248a7e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['text'] = val['text'].apply(clean_text)
<ipython-input-65-bb8034248a7e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [66]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Bidirectional, SimpleRNN, LSTM

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt

In [67]:
train['text']

,text
29527,None
7373,None
10326,None
23687,None
32062,None
...,...
31014,None
7392,None
1702,None
8881,None


In [68]:
tokenizer = Tokenizer()
# train['text'] = train['text'].str.split()

tokenizer.fit_on_texts(train['text'])

label_encoder = LabelEncoder()

def techniques(data):

    data['text'] = tokenizer.texts_to_sequences(data['text'])
    data['text'] = pad_sequences(data['text'], maxlen=300, padding='post')

    data['label'] = label_encoder.fit_transform(data['label'])
    return data

train = techniques(train)
val = techniques(val)
test = techniques(test)

AttributeError: 'NoneType' object has no attribute 'lower'

# Building the Classifier

In this step, you will design and build a NLP Classifier model to classify text data. Below is a breakdown of the key components you'll implement, but it's up to you to decide how to configure them based on your understanding and experimentation:

1. **Model Type**:
   You will use a Sequential model, which allows you to stack layers in a linear sequence.

2. **Input Layer**:
   Define the shape of the input data. Consider the dimensions of your padded sequences and set the input shape accordingly.

3. **Embedding Layer**:
   The embedding layer will convert input tokens (integers) into dense vector representations. You will need to determine the size of the input dimension (based on your vocabulary) and the output dimension (embedding size).

4. **Bidirectional Simple RNN/LSTM Layers**:
   You can add one or more recurrent layers. Consider using Bidirectional layers to capture contextual information from both directions (forward and backward). You can chose SimpleRNN/GRU/LSTM to perform this step.

5. **Dense Layers**:
   Add one or more fully connected (Dense) layers to process the output from the RNN/GRU/LSTM layers.

6. **Output Layer**:
   The output layer should match the type of classification task you're working on. Consider using appropriate activation function with appropriate number of units.

7. **Model Summary**:
   After defining your model architecture, print a summary to review the number of layers, types of layers, and total parameters.

8. **Model Compilation**:
   Finally, compile the model by selecting an optimizer, a loss function, and metrics.

In [69]:
model = Sequential([
    Input(shape=(300,)),
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=300),

    Bidirectional(SimpleRNN(64, return_sequences=True)),

    Bidirectional(SimpleRNN(32)),

    Dense(64, activation='relu'),

    Dense(1, activation='sigmoid')
])



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [70]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 300, 128)            │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 300, 128)            │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 64)                  │          10,304 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 39,361 (153.75 KB)

 Trainable params: 39,361 (153.75 KB)

 Non-trainable params: 0 (0.00 B)

In [71]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Defining Batch Size, Creating Datasets, and Training the Model

In this step, you will define the batch size, create TensorFlow Datasets for both training and testing, and train the model. The key elements to consider are outlined below, and it is up to you to choose the specific configurations based on your preferences and experimentation:

1. **Batch Size**:
   Select a batch size for training and testing. The batch size determines how many samples will be processed together in one forward and backward pass during training.

2. **Creating Datasets**:
   Use TensorFlow’s `Dataset.from_tensor_slices()` to create datasets from the training and testing data.

3. **Batching the Datasets**:
   Batch the datasets by grouping the data into batches of the specified size.

4. **Training the Model**:
   Train the model by fitting it on the training dataset for a specified number of epochs. You will also need to provide the validation data to monitor the model’s performance on unseen data during training.

5. **Tracking Training History**:
   During training, the model’s performance metrics (such as loss and accuracy) will be tracked over the epochs, and the results will be stored in the `history` object.

In [79]:
import tensorflow as tf
import tensorflow_datasets as tfds

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [80]:
train_dataset = tensorflow.data.Dataset.from_tensor_slices((train['text'], train['label']))

val_dataset = tensorflow.data.Dataset.from_tensor_slices((val['text'], val['label']))

test_dataset = tensorflow.data.Dataset.from_tensor_slices((test['text'], test['label']))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type NoneType).

In [78]:
batch_size = 32

train_dataset = tf.data.Dataset.from_tensor_slices((train['text'], train['label']))
test_dataset = tf.data.Dataset.from_tensor_slices((train['text'], train['label']))


train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)


history = model.fit(
    train_dataset,
    epochs=5,
    validation_data=test_dataset,
)


NameError: name 'tf' is not defined

In [74]:
history = model.fit(train['text'],train['label'], validation_data=(val['text'],val['label']), epochs=2,batch_size=32)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type NoneType).

# Model Evaluation

Once the model is trained, the next step is to evaluate its performance on the testing dataset.

1. **Evaluate the Model**:
   You will use the `evaluate()` method to assess the model’s performance on the test dataset.

2. **Testing Dataset**:
   Ensure that the testing dataset is properly prepared and batched, just like the training dataset.

4. **Loss Curve**:
   A loss curve plots the loss values for both the training and validation datasets over the epochs.

In [77]:
print(model.evaluate(test['text'], test['label']))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type NoneType).

In [ ]:
pd.DataFrame(history.history)[['val_accuracy','accuracy']].plot()

# Model Inference

In this step, you will use the trained model to make predictions on new, unseen data (inference). Here’s an outline of the key points:

1. **Create Test Sample**:
   Create a string to test your modelm the goal here is to give the model Before making predictions, ensure that the new data is preprocessed in the same way as the training data. This includes tokenization, padding, and any other transformations you applied during the data preprocessing step. The data can be single text to see the result of the prediction.

2. **Model Prediction**:
   Use the `predict()` method to feed new samples into the trained model and obtain predictions. The model will output probabilities or predicted class labels based on the type of classification task (binary or multi-class).

3. **Interpreting Predictions**:
   The model will return probabilities for each class.

In [ ]:
sample_text = ["معسكر تقنيات ادارة الازدحام"]

sample_sequence = tokenizer.texts_to_sequences([clean_text(sample_text[0])])

padded_sample_sequence = pad_sequences(sample_sequence, maxlen=300, padding='post')

predicted_label = model.predict(padded_sample_sequence)

print("Predicted label:", label_encoder.inverse_transform([predicted_label.argmax()]))

# Notebook Question:
- How did you handle text preprocessing? Why did you choose this approach?

- Why did you choose this model design?

- Why did you pick this number of layers or units for the model?

- Why did you select these evaluation methods?

- Does your model show signs of overfitting or underfitting? How do you know?

- What changes could you make to improve the model and fix overfitting or underfitting?

Answer Here: